In [11]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split as tts
import os

In [12]:
path = "archive"
save_path = "data"

new_lab = "Smoker" #needs to be exactly the same as a category from the csv file

In [13]:
isExist = os.path.exists(save_path)
if not isExist:
    os.makedirs(save_path)
    print("The new directory is created!")

In [14]:
data_pd = pd.read_csv(path + '/diabetes_binary_health_indicators_BRFSS2015.csv')
data_param = data_pd.columns
data = data_pd.values

In [6]:
labs = data[:,0]
data = data[:,1:]
X_train, X_test, y_train, y_test = tts(data, labs, test_size=0.1, shuffle=True)
data.shape, X_train.shape

((253680, 21), (228312, 21))

In [7]:
pca_dim2 = PCA(n_components=2) 
train_pca2 = pca_dim2.fit_transform(X_train, y_train)
test_pca2 = pca_dim2.transform(X_test)

pca_dim10 = PCA(n_components=10) 
train_pca10 = pca_dim10.fit_transform(X_train, y_train)
test_pca10 = pca_dim10.transform(X_test)

In [ ]:
pd.DataFrame(train_pca2).to_csv(save_path + '/trn_pca2.csv', index=False)
pd.DataFrame(test_pca2).to_csv(save_path + '/tst_pca2.csv', index=False)
pd.DataFrame(y_train).to_csv(save_path + '/trn_labs.csv', index=False)
pd.DataFrame(y_test).to_csv(save_path + '/tst_labs.csv', index=False)
pd.DataFrame(X_train).to_csv(save_path + '/trn_all.csv', index=False)
pd.DataFrame(X_test).to_csv(save_path + '/tst_all.csv', index=False)
pd.DataFrame(X_test).to_csv(save_path + '/tst_all.csv', index=False)
pd.DataFrame(data_param).to_csv(save_path + '/param.csv', index=False)

<h3>Smoking data</h3>

In [17]:
labs_ = data_pd[new_lab].values
dat_ = data_pd[data_pd.columns.drop(new_lab)].values

X_train_, X_test_, y_train_, y_test_ = tts(dat_, labs_, test_size=0.1, shuffle=True)

pca_dim2_ = PCA(n_components=2) 
train_pca2_ = pca_dim2_.fit_transform(X_train_, y_train_)
test_pca2_ = pca_dim2_.transform(X_test_)

pca_dim10_ = PCA(n_components=10) 
train_pca10_ = pca_dim10.fit_transform(X_train_, y_train_)
test_pca10_ = pca_dim10.transform(X_test_)

new_file = "data_"+new_lab
isExist = os.path.exists(new_file)
if not isExist:
    os.makedirs(new_file)
    print("The new directory is created!")

pd.DataFrame(train_pca2_).to_csv(new_file + '/trn_pca2.csv', index=False)
pd.DataFrame(test_pca2_).to_csv(new_file + '/tst_pca2.csv', index=False)
pd.DataFrame(train_pca10_).to_csv(new_file + '/trn_pca10.csv', index=False)
pd.DataFrame(test_pca10_).to_csv(new_file + '/tst_pca10.csv', index=False)
pd.DataFrame(y_train_).to_csv(new_file + '/trn_labs.csv', index=False)
pd.DataFrame(y_test_).to_csv(new_file + '/tst_labs.csv', index=False)
pd.DataFrame(X_train_).to_csv(new_file + '/trn_all.csv', index=False)
pd.DataFrame(X_test_).to_csv(new_file + '/tst_all.csv', index=False)
pd.DataFrame(X_test_).to_csv(new_file + '/tst_all.csv', index=False)
pd.DataFrame(data_param).to_csv(new_file + '/param.csv', index=False)